# Load Models and Generate the submission File

### This notebook will show how to
- load the checkpoints of our baseline models
- use our API to generate a submission .csv file
- use the .csv file to make a submission

We also show how
- we extract the ground truth responses from a dataset
- we use these ground truth responses and the submitted files to calculate the score of the live leaderboard of our competition

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

In [2]:
tier = 'train'
dataset_name = '20457-5-9-0'

filenames = ['./data/lurz2020/static20457-5-9-preproc0', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': True,
                 'batch_size': 128,
                 'exclude': None,
                 'file_tree': True,
                 'scale': 1,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

In [3]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'stack': -1,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_dilation': 1,
  'hidden_kern': 7,
  'hidden_channels': 64,
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': False,
  'use_avg_reg':False,
}

sota_model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

sota_model.load_state_dict(torch.load("./checkpoints/sota_model.pth"));

In [4]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 3,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_dilation': 1,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
              'linear': True
               }
ln_model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

ln_model.load_state_dict(torch.load("./checkpoints/ln_model.pth"));

# Generate the Submission File with our API

In [5]:
# import the API from the competition repo
from sensorium.utility import submission

In [6]:
# generate the submission file
submission.generate_submission_file(trained_model=sota_model, 
                                    test_dataloader=dataloaders["test"][dataset_name],
                                    data_key=dataset_name,
                                    path='./submission_files/sota_submission_file.csv',
                                    device="cuda")

File saved.


In [7]:
# identical procedure with the LN model
submission.generate_submission_file(trained_model=ln_model, 
                                    test_dataloader=dataloaders["test"][dataset_name], 
                                    path='./submission_files/ln_submission_file.csv',)

File saved.


### Inspect content of submission file

In [8]:
import pandas as pd
pd.read_csv('./submission_files/sota_submission_file.csv')

,trial_indices,image_ids,prediction,neuron_ids
0,126,2214,"[0.39909374713897705, 0.5152013301849365, 0.51...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
1,297,2214,"[0.3848317861557007, 0.5232589840888977, 0.504...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
2,597,2214,"[0.38958680629730225, 0.527167022228241, 0.512...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
3,852,2214,"[0.38263487815856934, 0.5397306084632874, 0.50...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
4,908,2214,"[0.38469237089157104, 0.5387403964996338, 0.50...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
...,...,...,...,...
994,2752,3487,"[0.0845494270324707, 0.14774799346923828, 0.27...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
995,3039,3487,"[0.08712136745452881, 0.1528913974761963, 0.25...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
996,4312,3487,"[0.08987849950790405, 0.16628813743591309, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
997,4380,3487,"[0.08792203664779663, 0.16572076082229614, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."


In [9]:
import pandas as pd
pd.read_csv('./submission_files/ln_submission_file.csv')

,trial_indices,image_ids,prediction,neuron_ids
0,126,2214,"[0.2543678283691406, 0.5152258276939392, 0.486...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
1,297,2214,"[0.25256574153900146, 0.49740755558013916, 0.4...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
2,597,2214,"[0.26838964223861694, 0.5102148652076721, 0.46...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
3,852,2214,"[0.2595486640930176, 0.5055870413780212, 0.500...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
4,908,2214,"[0.2610173225402832, 0.5303679704666138, 0.488...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
...,...,...,...,...
994,2752,3487,"[0.27398788928985596, 0.8061962127685547, 0.95...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
995,3039,3487,"[0.27083271741867065, 0.8950667977333069, 0.99...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
996,4312,3487,"[0.2699958086013794, 0.9457783699035645, 1.056...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
997,4380,3487,"[0.26997995376586914, 0.9041118621826172, 1.06...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."


# The submission process

- now that the .csv file is generated, the participant can make an entry to the competition
- all that is left to do is:
 - register a new user on http://sensorium2022.net/ with a valid email address
 - upload the .csv file
 - the competition score will be calculated automatically and will show up in the live leaderboard within a few seconds

- **Note**: Before June 15, our website does not accept submissions. 
The competition does start officially on June 15. On this date, we're releasing the competition dataset, and submissions will be open.

---

---

# How we calculate the competition scores behind the scenes

- we will withhold the ground truth neuronal responses to the test set images in the actual competition
- here we show 
 - how we extract the ground truth responses from the demo dataset (where the test set responses are present)
 - how the metrics of the competition are calculated

In [13]:
from sensorium import evaluate

In [14]:
# we load the dataset which contains the held-out "test" responses, and save them in the .csv format
# for the demo dataset that we provide here, these "test" responses are present

filename = ['./data/lurz2020/static20457-5-9-preproc0']
submission.generate_ground_truth_file(filename=filename,
                                      path='./ground_truth_files/ground_truth_file.csv')

### SOTA model results

In [15]:
ground_truth_file = './ground_truth_files/ground_truth_file.csv'
submission_file = './submission_files/sota_submission_file.csv'

In [16]:
out = evaluate(submission_file, ground_truth_file)

In [17]:
print("Results for the SOTA model:")
for metric, value in out.items():
    print(f"{metric}: {np.round(value, 3)}")

Results for the SOTA model:
Correlation (single trial): 0.302
Correlation (mean): 0.549
FEVE: 0.297


#### These scores are calcualted in the backend of our website
- we have two test sets, so these scores will be computed for our **live** test set, and our **final** test set
- the **live** scores will get published on the live leaderboard
- the **final** scores will be hidden, and we will release them to the public on Oct 22, after checking the scores carefully
- the **final** scores will then determine the winner of the competition

## Same process for the simple Baseline Model

### LN model results

In [18]:
ground_truth_file = './ground_truth_files/ground_truth_file.csv'
submission_file = './submission_files/ln_submission_file.csv'

In [19]:
out = evaluate(submission_file, ground_truth_file)

In [20]:
print("Results for the LN model:")
for metric, value in out.items():
    print(f"{metric}: {np.round(value,3)}")

Results for the LN model:
Correlation (single trial): 0.203
Correlation (mean): 0.365
FEVE: 0.131


---